In [1]:
import json
import onnx
import torch
from torch import nn

In [2]:
class ExampleModel(nn.Module):

    def __init__(self):
        super(ExampleModel, self).__init__()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 9)
        self.relu = nn.ReLU()

    def forward(self, x1, x2, x3):
        # Input x1 is a scalar
        # Input x2 is a vector with causality 'local' and 4 elements
        # Input x3 is a vector with causality 'countinuous' and 5 elements
        x4 = self.fc1(torch.cat((x2, x3), dim=0))
        x4 = self.fc2(x4)
        # Add dimension to x1
        x1 = x1.unsqueeze(0)
        # Concatenate x1, x4 and return
        x = torch.cat((x1, x4), dim=0)
        x = self.relu(x)
        return x

In [3]:
# Create three tensors
x1 = torch.tensor(1.0)
x2 = torch.randn(4)
x3 = torch.randn(5)

# Print tensors shape
print(x1.shape)
print(x2.shape)
print(x3.shape)

# Create the model
model = ExampleModel()

# Run the model
output = model(x1, x2, x3)
print(output)


torch.Size([])
torch.Size([4])
torch.Size([5])
tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6177, 0.0000,
        0.1084], grad_fn=<ReluBackward0>)


In [4]:
# Set the model to evaluation mode
model.eval()

# Save the model in ONNX format
torch.onnx.export(
    model,
    (x1, x2, x3),
    "example1.onnx",
    verbose=True,
    input_names=["scalar_input", "vector_input_local", "vector_input_continuous"],
    output_names=["output"],
)

# Load the model
model_name = "example1"
onnx_model = onnx.load(f"{model_name}.onnx")

# Check the model
onnx.checker.check_model(onnx_model)

# Add description to the model
onnx_model.graph.doc_string = "Example model with scalar and vector inputs"

# Add descrption to the input
onnx_model.graph.input[0].doc_string = "Scalar input"
onnx_model.graph.input[1].doc_string = "Vector input with causality local"
onnx_model.graph.input[2].doc_string = "Vector input with causality continuous"
onnx_model.graph.output[0].doc_string = "Output"

# Add metadata to the model
onnx_model.producer_name = "ExampleModel"
onnx_model.producer_version = "0.0.1"
onnx_model.domain = "example"
onnx_model.model_version = 1

# Save the model
onnx.save(onnx_model, f"{model_name}.onnx")


Exported graph: graph(%scalar_input : Float(requires_grad=0, device=cpu),
      %vector_input_local : Float(4, strides=[1], requires_grad=0, device=cpu),
      %vector_input_continuous : Float(5, strides=[1], requires_grad=0, device=cpu),
      %fc1.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %fc2.bias : Float(9, strides=[1], requires_grad=1, device=cpu),
      %onnx::MatMul_18 : Float(9, 64, strides=[1, 9], requires_grad=0, device=cpu),
      %onnx::MatMul_19 : Float(64, 9, strides=[1, 64], requires_grad=0, device=cpu)):
  %/Concat_output_0 : Float(9, strides=[1], requires_grad=0, device=cpu) = onnx::Concat[axis=0, onnx_name="/Concat"](%vector_input_local, %vector_input_continuous), scope: __main__.ExampleModel:: # /tmp/ipykernel_47028/529190988.py:13:0
  %/fc1/MatMul_output_0 : Float(64, strides=[1], device=cpu) = onnx::MatMul[onnx_name="/fc1/MatMul"](%/Concat_output_0, %onnx::MatMul_18), scope: __main__.ExampleModel::/torch.nn.modules.linear.Linear::fc1 # /home

Create and save the model description to be provided to ONNX2FMU.

In [7]:
model_description = {
    "name": "Example1",
    "description": "The model defines a simple example model with a scalar input and two vector inputs, one with 'local' variability and one with 'continuous' variability.",
    "FMIVersion": "2.0",
    "input": [
        {
            "name": "scalar_input",
            "description": "A scalar input to the model."
        },
        {
            "name": "vector_input_local",
            "description": "A vector of 'local' variability inputs.",
            "causality": "local"
        },
        {
            "name": "vector_input_continuous",
            "description": "A vector of 'continuous' variability inputs."
        }
    ],
    "output": [
        {
            "name": "output",
            "description": "The output array.",
            "names": ["Class1", "Class2", "Class3", "Class4", "Class5", "Class6", "Class7", "Class8", "Class9", "Class10"]
        }
    ]
}

# Save model description
with open(f"{model_name}Description.json", "w", encoding="utf-8") as f:
    json.dump(model_description, f, indent=4)